In [ ]:
# Potrzebne importy
import numpy as np
import pandas as pd

# Ćwiczenie 1

## Zadanie 1
Wczytanie pliku

In [ ]:
titanic = pd.read_csv('titanic.csv', na_values='?').set_index('id')
titanic.columns = ['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch',
                   'ticket', 'fare', 'cabin', 'embarked', 'boat', 'body',
                   'home.dest']

## Zadanie 2


In [ ]:
print(titanic.head(20))
print(titanic.info())

    pclass  survived                                              name  \
id                                                                       
1        1         1                      Allen,Miss. Elisabeth Walton   
2        1         1                     Allison,Master. Hudson Trevor   
3        1         0                       Allison,Miss. Helen Loraine   
4        1         0               Allison,Mr. Hudson Joshua Creighton   
5        1         0    Allison,Mrs. Hudson J C (Bessie Waldo Daniels)   
6        1         1                                Anderson,Mr. Harry   
7        1         1                  Andrews,Miss. Kornelia Theodosia   
8        1         0                             Andrews,Mr. Thomas Jr   
9        1         1      Appleton,Mrs. Edward Dale (Charlotte Lamson)   
10       1         0                            Artagaveytia,Mr. Ramon   
11       1         0                             Astor,Col. John Jacob   
12       1         1  Astor,Mrs. John 

1. wartości NA oznaczane są pytajnikami - w związku z tym dodaję do czytania pliku opcję na_values="?"
2. Niektóre kolumny są kategoryczne np. `survived`, `sex`
3. Po dodaniu na_values niektóre kolumny zmieniły swój typ na int64 i float64 - to dobrze bo na logikę powinny mieć taki typ, aczkolwiek i tak należałoby sprawdzić i zmienić typy niektórych kolumn 
4. Jest wiele brakujących danych (już na pierwszy rzut oka) - trzeba będzie się im przyjrzeć
5. Na titanicu było 1324 pasażerów, mamy 1309 wierszy danych co oznacza, że nasz zbiór jest w miare kompletny
6. Można pozbyć się wielu kolumn np. `body` (nr identyfikacyjny ciała), `name`, `ticket`, `home.dest`

## Zadanie 3

In [ ]:
print("Ilość brakujących danych wg kolumn")
print(titanic.isnull().sum(),"\n")
print("Procent brakujących danych w poszczególnych kolumnach")
print(titanic.isnull().mean())

Ilość brakujących danych wg kolumn
pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64 

Procent brakujących danych w poszczególnych kolumnach
pclass       0.000000
survived     0.000000
name         0.000000
sex          0.000000
age          0.200917
sibsp        0.000000
parch        0.000000
ticket       0.000000
fare         0.000764
cabin        0.774637
embarked     0.001528
boat         0.628724
body         0.907563
home.dest    0.430863
dtype: float64


Zmienne `age`, `fare`, `cabin`, `embarked`, `boat`, `body` i `home.dest` zawierają wiele nulli, niektóre powyżej 60% - jest do dosyć dużo, na pewno należy przyjrzeć się przyczynie (zadanie następne). Niektóre kolumny są całkowicie znane.

## Zadanie 4

In [ ]:
titanic["ageNull"] = np.where(titanic["age"].isnull(), 1, 0)
titanic["fareNull"] = np.where(titanic["fare"].isnull(), 1, 0)
titanic["cabinNull"] = np.where(titanic["cabin"].isnull(), 1, 0)
titanic["embarkedNull"] = np.where(titanic["embarked"].isnull(), 1, 0)
titanic["boatNull"] = np.where(titanic["boat"].isnull(), 1, 0)
titanic["bodyNull"] = np.where(titanic["body"].isnull(), 1, 0)
titanic["homedestNull"] = np.where(titanic["home.dest"].isnull(), 1, 0)
print(titanic.groupby("survived")["ageNull"].mean())
print(titanic.groupby("survived")["fareNull"].mean())
print(titanic.groupby("survived")["cabinNull"].mean())
print(titanic.groupby("survived")["embarkedNull"].mean())
print(titanic.groupby("survived")["boatNull"].mean())
print(titanic.groupby("survived")["bodyNull"].mean())
print(titanic.groupby("survived")["homedestNull"].mean())

survived
0    0.234858
1    0.146000
Name: ageNull, dtype: float64
survived
0    0.001236
1    0.000000
Name: fareNull, dtype: float64
survived
0    0.873918
1    0.614000
Name: cabinNull, dtype: float64
survived
0    0.000
1    0.004
Name: embarkedNull, dtype: float64
survived
0    0.988875
1    0.046000
Name: boatNull, dtype: float64
survived
0    0.850433
1    1.000000
Name: bodyNull, dtype: float64
survived
0    0.508035
1    0.306000
Name: homedestNull, dtype: float64


1. kolumna `age` - MAR, ponieważ więcej danych brakuje dla osób które zginęły, prawdopodobnie te dane nie były zbierane przy kupnie biletu a później nie dotarto do szczegółowych informacji o tych osobach
2. kolumna `fare` - MNAR, ponieważ brakuje informacji tylko dla osób które nie przeżyły, jest to jednak mniej niż 5% więc nie przejmujemy się tym
3. kolumna `cabin` - MAR, ponieważ więcej informacji brakuje dla osób które nie przeżyły, stosunek procentowy nie jest jednak wielki; numer kabiny nie jest szczególnie istotny, jednak da się z niego wyciągnąć informację o pokładzie co jest informacją bardzo istotną.
4. kolumna `embarked` - MCAR, mniej niż 5%, co prawda więcej informacji brakuje dla osób z jednej grupy (tych które przeżyły) ale jest to na tyle mała różnica że możemy uznać że danych brakuje całkowicie losowo
5. kolumna `boat` - MNAR, osoby którym nie udało się dostać na szalupę nie będą miały jej zapisanej, 4% osób które przeżyły również nie ma jej wpisanej - prawdopodobnie zostały wyłowione z wody
6. kolumna `body` - MNAR, nie wszyscy zginęli, więc nie zostały im nadane numery identyfikacyjne dla ciał; wielu ciał nie odnaleziono
7. kolumna `home.dest` - MAR - brakuje danych zarówno dla osób które przeżyły jak i nie, więcej danych brakuje jednak dla osób które nie przeżyły - nie można ich było dopytać o to; więcej danych brakuje dla osób podróżujących 3 klasą - jest to efekt zarówno faktu że większość z nich nie przeżyła jak i faktu że takimi osobami przed podróżą niezbyt mocno się interesowano

## Zadanie 5
1. Dla cech w których brakuje mniej niż 5% danych - można spokojnie je usunąć gdyż nie jest to zbyt dużo wartości żeby się nimi przejmować. (`fare` i `embarked`).
2. Dla kolumny `age` - można wypełnić średnią, np. z uwzględnieniem płci i klasy
3. Dla kolumny `cabin` - Można wypełnić losowymi danymi dla osób z tej samej klasy (zakładamy że kabiny dla poszczególnych klas są na tym samym pokładzie statku a to jest prawdopodobnie najważniejsza informacja związana z przeżyciem - osoba mieszkająca na najniższym pokładzie ma w środku nocy mniejsze szanse na wydostanie się na zewnątrz niż osoba będąca na pokładzie blisko poziomu morza).
4. Dla kolumny `boat` - możemy uzupełnić pewną zadedykowaną wartością oznaczającą, że ta osoba nie dostała się na szalupę ratunkową, np. 0
5. Dla kolumny `body` - tutaj również nie da się tego w żaden sensowny sposób wypełnić, albo należy dać dedykowane wartości oznaczone "not found" - dla osób zmarłych i "not relevant" dla osób które przeżyły. Można również całkowicie pozbyć się tej kolumny - nie wnosi ona zbyt wielu informacji, mogłaby powodować przetrenowanie modelu.
6. Dla kolumny `home.dest` - można wypełnić losowymi innymi danymi lub również pozbyć się tej kolumny, nie jest ona szczególnie ważna

# Ćwiczenie 2

## Zadanie 1

In [ ]:
# 'pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch','ticket', 'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'
print(f'Liczba etykiet zmiennej "pclass" (klasa): {len(titanic.pclass.unique())}')
print(f'Liczba etykiet zmiennej "survived" (czy przeżył): {len(titanic.survived.unique())}')
print(f'Liczba etykiet zmiennej "name" (imię): {len(titanic.name.unique())}')
print(f'Liczba etykiet zmiennej "sex" (płeć): {len(titanic.sex.unique())}')
print(f'Liczba etykiet zmiennej "sibsp": {len(titanic.sibsp.unique())}')
print(f'Liczba etykiet zmiennej "parch": {len(titanic.parch.unique())}')
print(f'Liczba etykiet zmiennej "ticket" (nr biletu): {len(titanic.ticket.unique())}')
print(f'Liczba etykiet zmiennej "cabin" (nr kabiny): {len(titanic.cabin.unique())}')
print(f'Liczba etykiet zmiennej "embarked" (gdzie wsiadł): {len(titanic.embarked.unique())}')
print(f'Liczba etykiet zmiennej "boat" (nr szalupy ratunkowej): {len(titanic.boat.unique())}')
print(f'Liczba etykiet zmiennej "body" (nr identyfikacyjny ciała): {len(titanic.body.unique())}')
print(f'Liczba etykiet zmiennej "home.dest" (cel podróży): {len(titanic["home.dest"].unique())}')

Liczba etykiet zmiennej "pclass" (klasa): 3
Liczba etykiet zmiennej "survived" (czy przeżył): 2
Liczba etykiet zmiennej "name" (imię): 1307
Liczba etykiet zmiennej "sex" (płeć): 2
Liczba etykiet zmiennej "sibsp": 7
Liczba etykiet zmiennej "parch": 8
Liczba etykiet zmiennej "ticket" (nr biletu): 929
Liczba etykiet zmiennej "cabin" (nr kabiny): 187
Liczba etykiet zmiennej "embarked" (gdzie wsiadł): 4
Liczba etykiet zmiennej "boat" (nr szalupy ratunkowej): 28
Liczba etykiet zmiennej "body" (nr identyfikacyjny ciała): 122
Liczba etykiet zmiennej "home.dest" (cel podróży): 370


## Zadanie 2

In [ ]:
print(f'Ilość wszystkich pasażerów: {len(titanic)}')

Ilość wszystkich pasażerów: 1309


## Zadanie 3
Wnioski:
* Mając 1309 pasażerów widzimy, że w dwóch przypadkach powtórzyło się to samo imię i nazwisko, lub dwukrotnie zapisano tą samą osobę
* Z poprzednich zadań wiemy, że posiadamy wszystkie informacje o biletach, więc średnio na jednym bilecie płynęło 1,4 osoby. Prawdopodobnie osoby z tego samego biletu to najbliższa rodzina i albo obie osoby przeżyły albo żadna - może to być ważna informacja.
* Mamy dwie zmienne które można uznać za binarne - `survived` (wartości 0 i 1) oraz `sex` (wartości female i male)
* Zmienne o małej kardynalności, które można bez problemu zapisać używając one-hot encodingu to: `pclass`, `sibsp`, `parch` i `embarked`. Można też do tego zbioru zaliczyć cechy `survived` i `sex`.
* Zmienne o dużej kardynalności to wszystkie pozostałe, przy czym zmienną `name` nie powinnyśmy i tak się zajmować, gdyż zawiera (praktycznie) unikatowe dane dla każdego pasażera.




## Zadanie 4

In [ ]:
titanic['cabin'] = titanic.cabin.astype('string')
unique, counts = np.unique(titanic[~titanic['cabin'].isnull()].cabin,
                           return_counts=True)
print(f'Ilość unikatowych etykiet: {len(unique)}')

Ilość unikatowych etykiet: 186


NumPy array to tablica, może być wielowymiarowa, w której wszystkie wartości są tego samego typu. Jeśli część danych byłaby typu int, a część float to nastąpi rzutowanie na bardziej ogólny typ danych (w tym przypadku na float). Z tego powodu obliczenia na ndarray będą szybsze niż na DataFrame, ale mogą być trudniejsze "myśleniowo". W obiekcie typu DataFrame każda kolumna może mieć inny typ, dane wyświetlane sa również automatycznie w sposób bardziej przejrzysty i są łatwiejsze do zrozumienia.

## Zadanie 5

In [ ]:
# Dla osób które nie posiadają przypisanej kabiny ustawiamy wartość kabiny 'N'
titanic['cabinReduced'] = titanic.cabin.fillna('N').str[0]
titanic[['cabin', 'cabinReduced']].head(20)

,cabin,cabinReduced
id,,
1,B5,B
2,C22 C26,C
3,C22 C26,C
4,C22 C26,C
5,C22 C26,C
6,E12,E
7,D7,D
8,A36,A
9,C101,C


## Zadanie 6

In [ ]:
n_original = len(titanic.cabin.unique())
n_reduced = len(titanic.cabinReduced.unique())
print(f'Liczba etykiet zmiennej "cabin" (nr kabiny): {n_original}')
print(f'Liczba etykiet zmiennej "cabinReduced" (nr kabiny po redukcji): {n_reduced}')
print(f'Zredukowano kardynalność zbioru o {np.round((n_original-n_reduced)/n_original*100,2)}%')

Liczba etykiet zmiennej "cabin" (nr kabiny): 187
Liczba etykiet zmiennej "cabinReduced" (nr kabiny po redukcji): 9
Zredukowano kardynalność zbioru o 95.19%


# Zadanie 7

* Tak jak pisałam w zadaniach 4 i 5 z poprzedniego ćwiczenia - pierwsza litera prawdopodobnie oznacza, pokład co jest bardzo istotną informacją z punktu widzenia osób próbujących wydostać się z tonącego statku. 
* Zmniejszenie ilości etykiet o 95% jest bardzo korzystne, a w tym przypadku dodatkowo wydaje się, że nie tracimy przy tym zbyt wielu informacji.
* W przyszłych analizach, po użyciu one-hot encodingu, możemy zaoszczędzić ogromną ilość miejsca, ponieważ dostaniemy jedynie 7 lub 8 nowych kolumn zamiast około 180.
* Należy jednak pamiętać, że wciąż prawie 70% danych w tej kolumnie jest wartościami NaN i należy wcześniej poradzić sobie z tymi brakującymi danymi (zadanie 5 punkt 3)
* Negatywnym skutkiem jest na pewno utracenie części danych, bo np. kolejne litery/cyfry mogłyby nam powiedzieć jak daleko kabina znajdowała się od schodów za pomocą których można się ewakuować, ale tak duże zmniejszenie kardynalności tego zbioru to rekompensuje - wciąż mamy wiele innych danych do modelowania których możemy użyć tym bardziej, że nie znamy wielu danych w tej kolumnie

In [ ]:
titanic.to_csv('titanic_better.csv')